# Getting started

In this notebook, we illustrate how to use the Neural News Recommendation with Multi-Head Self-Attention ([NRMS](https://aclanthology.org/D19-1671/)). The implementation is taken from the [recommenders](https://github.com/recommenders-team/recommenders) repository. We have simply stripped the model to keep it cleaner.

We use a small dataset, which is downloaded from [recsys.eb.dk](https://recsys.eb.dk/). All the datasets are stored in the folder path ```~/ebnerd_data/*```.

## Load functionality

In [1]:
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import tensorflow as tf
import polars as pl
import datetime

from ebrec.utils._constants import *

from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    add_prediction_scores,
    add_known_user_column,
    truncate_history,
)
from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
from ebrec.utils._articles import convert_text2encoding_with_transformers
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._articles import create_article_id_to_value_mapping
from ebrec.utils._nlp import get_transformers_word_embeddings
from ebrec.utils._python import write_submission_file, rank_predictions_by_score

from ebrec.models.newsrec.dataloader import NRMSDataLoader
from ebrec.models.newsrec.model_config import hparams_nrms
from ebrec.models.newsrec import NRMSModel #, NRMSWrapper

2024-12-19 16:29:56.254649: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-19 16:29:56.300213: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 16:29:56.300248: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 16:29:56.300275: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 16:29:56.309634: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [2]:
# List all physical devices
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

physical_devices = tf.config.list_physical_devices()
print("Available devices:", physical_devices)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2024-12-19 16:29:59.537088: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Load dataset

### Generate labels
We sample a few just to get started. For testset we just make up a dummy column with 0 and 1 - this is not the true labels.

In [3]:
PATH = Path("/dtu/blackhole/14/155764/DeepL-Project-Corn2/ebnerd-benchmark-copy/ebnerd_data").expanduser()
#
DATASPLIT = "ebnerd_small"
DUMP_DIR = Path("ebnerd_predictions")
DUMP_DIR.mkdir(exist_ok=True, parents=True)

History size can often be a memory bottleneck; if adjusted, the NRMS hyperparameter ```history_size``` must be updated to ensure compatibility and efficient memory usage

In [4]:
HISTORY_SIZE = 20
hparams_nrms.history_size = HISTORY_SIZE

In [5]:
# We just want to load the necessary columns
COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_IMPRESSION_TIMESTAMP_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
]
# This notebook is just a simple 'get-started'; we down sample the number of samples to just run quickly through it.
FRACTION = 0.01

In this example we sample the dataset, just to keep it smaller. We'll split the training data into training and validation 

In [6]:
def ebnerd_from_path(
    path: Path,
    history_size: int = 30,
    padding: int = 0,
    user_col: str = DEFAULT_USER_COL,
    history_aids_col: str = DEFAULT_HISTORY_ARTICLE_ID_COL,
) -> pl.DataFrame:
    """
    Load ebnerd - function
    """
    df_history = (
        pl.scan_parquet(path.joinpath("history.parquet"))
        .select(user_col, history_aids_col)
        .pipe(
            truncate_history,
            column=history_aids_col,
            history_size=history_size,
            padding_value=padding,
            enable_warning=False,
        )
    )
    df_behaviors = (
        pl.scan_parquet(path.joinpath("behaviors.parquet"))
        .collect()
        .pipe(
            slice_join_dataframes,
            df2=df_history.collect(),
            on=user_col,
            how="left",
        )
    )
    return df_behaviors

In [7]:
df = (
    ebnerd_from_path(
        PATH.joinpath(DATASPLIT, "train"),
        history_size=HISTORY_SIZE,
        padding=0,
    )
    .select(COLUMNS)
    .pipe(
        sampling_strategy_wu2019,
        npratio=4,
        shuffle=True,
        with_replacement=True,
        seed=123,
    )
    .pipe(create_binary_labels_column)
    .sample(fraction=FRACTION)
)

dt_split = pl.col(DEFAULT_IMPRESSION_TIMESTAMP_COL).max() - datetime.timedelta(days=1)
df_train = df.filter(pl.col(DEFAULT_IMPRESSION_TIMESTAMP_COL) < dt_split)
df_validation = df.filter(pl.col(DEFAULT_IMPRESSION_TIMESTAMP_COL) >= dt_split)

print(f"Train samples: {df_train.height}\nValidation samples: {df_validation.height}")
df_train.head(2)

Train samples: 1995
Validation samples: 347


user_id,impression_id,impression_time,article_id_fixed,article_ids_clicked,article_ids_inview,labels
u32,u32,datetime[μs],list[i32],list[i64],list[i64],list[i8]
586962,139550864,2023-05-19 16:05:47,"[9768860, 9767534, … 9770867]",[9772029],"[9746360, 9768583, … 9772029]","[0, 0, … 1]"
709317,17296567,2023-05-19 05:31:06,"[9768829, 9767637, … 9770594]",[9771627],"[9771627, 9099237, … 9771119]","[1, 0, … 0]"


### Test set
We'll use the validation set, as the test set.

In [10]:
COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_IMPRESSION_TIMESTAMP_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
]

In [34]:
PATH= Path("/dtu/blackhole/14/155764/DeepL-Project-Corn2/ebnerd-benchmark-copy/ebnerd_data").expanduser()
datasplit="ebnerd_testset"

df_test = (
    ebnerd_from_path(
        PATH.joinpath(datasplit, "test"),
        # PATH.joinpath(DATASPLIT, "validation"),
        history_size=HISTORY_SIZE,
        padding=0,
    )
    .select(COLUMNS)
    .pipe(create_binary_labels_column)
    # .sample(fraction=FRACTION)
)

KeyboardInterrupt: 

## Load articles

In [20]:
df_articles = pl.read_parquet(PATH.joinpath(DATASPLIT, "articles.parquet"))
df_articles.head(2)

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var i…","""Politiet frygt…",2023-06-29 06:20:33,false,"""Sagen om den ø…",2006-08-31 08:06:45,[3150850],"""article_defaul…","""https://ekstra…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars …","""Biografgængern…",2023-06-29 06:20:35,false,"""Vatikanet har …",2006-05-21 16:57:00,[3006712],"""article_defaul…","""https://ekstra…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""


## Init model using HuggingFace's tokenizer and wordembedding
In the original implementation, they use the GloVe embeddings and tokenizer. To get going fast, we'll use a multilingual LLM from Hugging Face. 
Utilizing the tokenizer to tokenize the articles and the word-embedding to init NRMS.


In [ ]:
TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-base"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

# LOAD HUGGINGFACE:
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# We'll init the word embeddings using the
word2vec_embedding = get_transformers_word_embeddings(transformer_model)
#
df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(
    df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
# =>
article_mapping = create_article_id_to_value_mapping(
    df=df_articles, value_col=token_col_title
)

/dtu/blackhole/14/155764/DeepL-Project-Corn2/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Initiate the dataloaders
In the implementations we have disconnected the models and data. Hence, you should built a dataloader that fits your needs.

Note, with this ```NRMSDataLoader``` the ```eval_mode=False``` is meant for ```model.model.fit()``` whereas ```eval_mode=True``` is meant for ```model.scorer.predict()```. 

In [22]:
BATCH_SIZE = 32

train_dataloader = NRMSDataLoader(
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=BATCH_SIZE,
)
val_dataloader = NRMSDataLoader(
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=BATCH_SIZE,
)

## Train the model


In [23]:
# List all physical devices
physical_devices = tf.config.list_physical_devices()
print("Available devices:", physical_devices)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


Initiate the NRMS-model:

In [24]:
import os
MODEL_NAME = "NRMS"
LOG_DIR = f"downloads/runs/{MODEL_NAME}"
WEIGHTS_DIR = f"downloads/data/state_dict/{MODEL_NAME}"
MODEL_WEIGHTS = f"{WEIGHTS_DIR}/weights.weights.h5"

os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(WEIGHTS_DIR, exist_ok=True)

# CALLBACKS
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_WEIGHTS, save_best_only=True, save_weights_only=True, verbose=1
)

hparams_nrms.history_size = HISTORY_SIZE
model = NRMSModel(
    hparams=hparams_nrms,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)
hist = model.model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=1,
    callbacks=[early_stopping, modelcheckpoint]#tensorboard_callback
)
_ = model.model.load_weights(filepath=MODEL_WEIGHTS)


63/63 [==============================] - ETA: 0s - loss: 1.6039
Epoch 1: val_loss improved from inf to 1.61417, saving model to downloads/data/state_dict/NRMS/weights.weights.h5
63/63 [==============================] - 74s 1s/step - loss: 1.6039 - val_loss: 1.6142


### Callbacks
We will add some callbacks to model training.

In [25]:
# Tensorboard:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=LOG_DIR,
    histogram_freq=1,
)

# Earlystopping:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_auc",
    mode="max",
    patience=3,
    restore_best_weights=True,
)

# ModelCheckpoint:
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_WEIGHTS,
    monitor="val_auc",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
)

# Learning rate scheduler:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc",
    mode="max",
    factor=0.2,
    patience=2,
    min_lr=1e-6,
)

callbacks = [tensorboard_callback, early_stopping, modelcheckpoint, lr_scheduler]

In [ ]:
USE_CALLBACKS = True
EPOCHS = 1

hist = model.model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=EPOCHS,
    callbacks=callbacks if USE_CALLBACKS else [],
)

63/63 [==============================] - ETA: 0s - loss: 1.5528

2024-12-19 16:46:19.617716: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 46080368640 exceeds 10% of free system memory.
2024-12-19 16:46:28.098013: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 46080368640 exceeds 10% of free system memory.


63/63 [==============================] - 83s 1s/step - loss: 1.5528 - val_loss: 1.6268 - lr: 1.0000e-04


In [27]:
if USE_CALLBACKS:
    _ = model.model.load_weights(filepath=MODEL_WEIGHTS)

# Example how to compute some metrics:

In [28]:
BATCH_SIZE_TEST = 16

test_dataloader = NRMSDataLoader(
    behaviors=df_test,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=BATCH_SIZE_TEST,
)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/dtu/blackhole/14/155764/DeepL-Project-Corn2/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3880890/1779326870.py", line 3, in <module>
    test_dataloader = NRMSDataLoader(
                      ^^^^^^^^^^^^^^^
  File "<string>", line 13, in __init__
  File "/dtu/blackhole/14/155764/DeepL-Project-Corn2/ebnerd-benchmark-copy/src/ebrec/models/newsrec/dataloader.py", line 44, in __post_init__
    self.X, self.y = self.load_data()
                     ^^^^^^^^^^^^^^^^
  File "/dtu/blackhole/14/155764/DeepL-Project-Corn2/ebnerd-benchmark-copy/src/ebrec/models/newsrec/dataloader.py", line 58, in load_data
    y = self.behaviors[self.labels_col]
        ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "/dtu/blackhole/14/155764/DeepL-Project-Corn2/.venv/lib/python3.11/site-packages/polars/dataframe/frame.py", line 1698, in __getitem__
    

In [ ]:
pred_test = model.scorer.predict(test_dataloader)

153/153 [==============================] - 122s 794ms/step


## Add the predictions to the dataframe

In [ ]:
df_test = add_prediction_scores(df_test, pred_test.tolist())
df_test.head(2)

user_id,impression_id,impression_time,article_id_fixed,article_ids_clicked,article_ids_inview,labels,scores
u32,u32,datetime[μs],list[i32],list[i32],list[i32],list[i8],list[f64]
983098,202907971,2023-05-28 20:43:01,"[9775701, 9775983, … 9778939]",[9766514],"[9785349, 9785209, … 9780736]","[0, 0, … 0]","[0.479354, 0.453712, … 0.475159]"
513401,569008456,2023-06-01 05:09:38,"[9778804, 9778219, … 9778939]",[9790784],"[9484153, 9790825, … 9527795]","[0, 0, … 0]","[0.393523, 0.518457, … 0.357469]"


### Compute metrics

In [ ]:
metrics = MetricEvaluator(
    labels=df_test["labels"].to_list(),
    predictions=df_test["scores"].to_list(),
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

<MetricEvaluator class>: 
 {
    "auc": 0.5243153252658638,
    "mrr": 0.32957706898653777,
    "ndcg@5": 0.36306189378701875,
    "ndcg@10": 0.44802213039224403
}

## Make submission file

In [ ]:
df_test = df_test.with_columns(
    pl.col("scores")
    .map_elements(lambda x: list(rank_predictions_by_score(x)))
    .alias("ranked_scores")
)
df_test.head(2)

user_id,impression_id,impression_time,article_id_fixed,article_ids_clicked,article_ids_inview,labels,scores,ranked_scores
u32,u32,datetime[μs],list[i32],list[i32],list[i32],list[i8],list[f64],list[i64]
983098,202907971,2023-05-28 20:43:01,"[9775701, 9775983, … 9778939]",[9766514],"[9785349, 9785209, … 9780736]","[0, 0, … 0]","[0.479354, 0.453712, … 0.475159]","[2, 5, … 3]"
513401,569008456,2023-06-01 05:09:38,"[9778804, 9778219, … 9778939]",[9790784],"[9484153, 9790825, … 9527795]","[0, 0, … 0]","[0.393523, 0.518457, … 0.357469]","[8, 1, … 11]"


This is using the validation, simply add the testset to your flow.

In [ ]:
write_submission_file(
    impression_ids=df_test[DEFAULT_IMPRESSION_ID_COL],
    prediction_scores=df_test["ranked_scores"],
    path=DUMP_DIR.joinpath("predictions.txt"),
    filename_zip=f"smoolasfk_predictions-{MODEL_NAME}.zip",
)

2446it [00:00, 22910.17it/s]

Zipping ebnerd_predictions/predictions.txt to ebnerd_predictions/smoolasfk_predictions-NRMS.zip


# DONE 🚀

In [35]:
import pandas as pd

# Path to the parquet file
file_path = '/dtu/blackhole/14/155764/DeepL-Project-Corn2/ebnerd-benchmark-copy/ebnerd_data/ebnerd_small/validation/behaviors.parquet'
# Load the parquet file
df = pd.read_parquet(file_path)
df.head(2)
# # Look for the specified Impression IDs
# impression_ids = [202907971, 6451339]
# filtered_data = df[df['impression_id'].isin(impression_ids)]
# filtered_data

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,96791,NaN,2023-05-28 04:21:24,9.0,NaN,2,"[9783865, 9784591, 9784679, 9784696, 9784710]",[9784696],22548,False,NaN,NaN,NaN,False,142,72.0,100.0
1,96798,NaN,2023-05-28 04:31:48,46.0,NaN,2,"[9782884, 9783865, 9782726, 9695098, 9782806, ...",[9784281],22548,False,NaN,NaN,NaN,False,143,16.0,28.0


In [36]:
file_path = '/dtu/blackhole/14/155764/DeepL-Project-Corn2/ebnerd-benchmark-copy/ebnerd_data/ebnerd_testset/test/behaviors.parquet'
# Load the parquet file
df = pd.read_parquet(file_path)
df.head(2)

,impression_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,is_beyond_accuracy
0,6451339,2023-06-05 15:02:49,8.0,NaN,2,"[9796527, 7851321, 9798805, 9795150, 9531110, ...",35982,False,NaN,NaN,NaN,False,388,False
1,6451363,2023-06-05 15:03:56,20.0,NaN,2,"[9798532, 9791602, 9798975, 9791334, 9793856, ...",36012,False,NaN,NaN,NaN,False,804,False
